<a href="https://colab.research.google.com/github/DarioMameli/DL_Assignments/blob/main/DL_Assignment3_Clasification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DL Assignment 3
## Group 4
**AUTHORS**:
*   Dario Mameli [dario.mameli@ugent.be]
*   Àngel Masip LLopis [angel.masipllopis@ugent.be]
*   Michele Russo [michele.russo@ugent.be]



This notebook is to be intended as both report and code.

Running on google colab is suggested to make sure all required libraries and packages are present.

#GitHub

Let's clone the repo with the dataset

In [1]:
!git clone https://github.com/dkdk22/DL_Ass3
import sys
# Add the repository to the path
sys.path.insert(1, '/content/DL_Ass3/')

Cloning into 'DL_Ass3'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (6/6), 22.59 MiB | 14.37 MiB/s, done.


# Setup

Firstly let's setup the workspace

## Pip Installs

In [2]:
# Install the latest version of TensorFlow, which includes Keras (tf.keras)
#!pip install tensorflow --upgrade
!pip install keras --upgrade --quiet
!pip install keras-tuner --upgrade
!pip install PrettyTable

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 15.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.1.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.3 MB/s eta 0:00:00


## Packages

Let's import all the necessary functions and packages

In [12]:
from dl_utils import plot_history
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import keras
from keras import regularizers
import matplotlib.gridspec as gridspec
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
import nltk
from nltk.corpus import stopwords
import keras_tuner as kt
from keras import backend as backend
from keras.layers import (
    Input,
    Activation,
    BatchNormalization,
    Conv2D,
    Conv2DTranspose,
    Dense,
    Dropout,
    Flatten,
    GlobalAveragePooling2D,
    MaxPooling2D,
    UpSampling2D
)
from keras.optimizers import SGD, Adam

import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
import json
import zipfile
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, f1_score, precision_score, recall_score
import gdown
import zipfile
import os
from keras import Model, ops
import joblib
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score
from sklearn.ensemble import GradientBoostingClassifier
from nltk.stem import PorterStemmer
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras.applications as app
from keras.layers import LSTM, Dense, Embedding, Input, GRU, Bidirectional
from sklearn.preprocessing import StandardScaler
import gc

from prettytable import PrettyTable

import zipfile
from skimage.segmentation import mark_boundaries
from sklearn.preprocessing import minmax_scale

import warnings
warnings.filterwarnings('ignore')

#Global variables

In [4]:
SEED =42
TRAIN_BASELINE = False
TRAIN_MODEL = True

# Load data

Load the dataset

In [5]:
zip_file_path = 'DL_Ass3/review_553850.zip'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref: #Extract the file
    zip_ref.extractall()

with open('review_553850.json', 'r') as json_file: # Open the json file
    data = json.load(json_file)

X = []
y = []

for id, review in data["reviews"].items():
    review_text = review["review"]
    funny_votes = review["votes_funny"]
    X.append(review_text)
    y.append(funny_votes)

##Data preprocessing

We are going to do the same preprocessing as in the other notebook. We are going to eliminate stopwords and also we are going to use steeming, tokenize and pad the sentences.


In [6]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
# Remove stopwords, convert to lowercase and stemming
X_prepro = []
for text in X:
    words = text.split()

    filtered_words = [word.lower() for word in words if word.lower() not in stop_words]
    stemmed_tokens = [stemmer.stem(word) for word in filtered_words]
    preprocessed_text = ' '.join(stemmed_tokens)
    X_prepro.append(preprocessed_text)
# Remove the reviews with 0 words
X_filtered = []
y_filtered = []
for review, label in zip(X_prepro, y):
    if len(review.split()) > 0:
        X_filtered.append(review)
        y_filtered.append(label)
print("Length of the dataset: "+str(len(X_filtered)))
# Tokenize
num_words=10000
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(X_filtered)
X_sequences = tokenizer.texts_to_sequences(X_filtered)
# Initialize variables to store information about the longest sequence
max_length = 0
max_index = -1

# Iterate through the list of sequences to have the longes sequence
for i, sequence in enumerate(X_sequences):
    length = len(sequence)
    # Update max_length and max_index if the current sequence is longer
    if length > max_length:
        max_length = length
        max_index = i

# Retrieve the longest sequence
longest_sequence = X_sequences[max_index]

# Print the longest sequence
print("The longest sequence:")
print(longest_sequence)
print(len(longest_sequence))
# Padding
max_sequence_length = len(longest_sequence)
padded_sequences = pad_sequences(X_sequences, maxlen=max_sequence_length)
# Convert to np arrays
scaler = StandardScaler()
normalized_data = scaler.fit_transform(padded_sequences)
X = np.array(normalized_data)
print(X.shape)



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Length of the dataset: 188843
The longest sequence:
[169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 154, 169, 15

In [7]:
del stop_words
del stemmer
del X_prepro
del text
del words
del filtered_words
del stemmed_tokens
del preprocessed_text
del X_filtered
del review
del num_words
del tokenizer
del X_sequences
del max_length
del max_index
del sequence
del longest_sequence
del padded_sequences
del scaler
del normalized_data
gc.collect()

0

#Binary classification problem

We can see that this is a really difficoult task to learn because most of the reviews have 0 funny_votes and some of them have a really high value of funny_votes. That's why we are going to change the approach and convert the problem to a binary classification problem, that should be easier to learn to our model.

We are going to convert it by fixing an number of funny_votes N, treating a review as funny or not funny depending on whether it has more or fewer votes than N. In our case N is going to be 1 because of the unbalancement of the dataset.

In [8]:
y = [0 if num == 0 else 1 for num in y_filtered]
y = np.array(y)
counts = [0,0]
counts[0] = np.sum(data == 0)
counts[1] = np.sum(data == 1)
del y_filtered
gc.collect()

0

##Baseline Model

In [9]:
baseline_model = GradientBoostingClassifier()

##Best model for classification problem

In [39]:
class HyperModelBuilderBinary(kt.HyperModel):
    def __init__(self, max_sequence_length, loss_function, max_features):
        self.max_sequence_length = max_sequence_length
        self.loss_function = loss_function
        self.max_features = max_features

    def build_lstm_model(self, hp):
        model = Sequential()
        model.add(Input(shape=(self.max_sequence_length,)))
        model.add(Embedding(input_dim=self.max_features, output_dim=hp.Int('embedding_dim', min_value=128, max_value=128, step=32)))  #fix embedding size?
        model.add(LSTM(units=hp.Int('units', min_value=64, max_value=256, step=32), #https://keras.io/api/layers/recurrent_layers/lstm/
                       dropout=hp.Float('dropout', min_value=0.2, max_value=0.4, step=0.2),
                       recurrent_dropout=hp.Float('dropout', min_value=0.2, max_value=0.4, step=0.2), #reduce two dropouts
                       input_shape=(self.max_sequence_length, 1)))
        model.add(Dense(1, activation="sigmoid"))
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])),
                      loss=self.loss_function,
                      metrics=["accuracy"])
        return model

    def build_bidirectional_lstm_model(self, hp):
        model = Sequential()
        model.add(Input(shape=(self.max_sequence_length,)))
        model.add(Embedding(input_dim=self.max_features, output_dim=hp.Int('embedding_dim', min_value=128, max_value=128, step=32)))
        model.add(Bidirectional(LSTM(units=hp.Int('units', min_value=64, max_value=256, step=32),
                                     dropout=hp.Float('dropout', min_value=0.2, max_value=0.4, step=0.2),
                                     recurrent_dropout=hp.Float('dropout', min_value=0.2, max_value=0.4, step=0.2),
                                     input_shape=(self.max_sequence_length, 1))))
        model.add(Dense(1, activation="sigmoid"))

        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])),
                      loss=self.loss_function,
                      metrics=["accuracy"])
        return model


    def build_gru_model(self, hp):
        model = Sequential()
        model.add(Input(shape=(self.max_sequence_length,)))
        model.add(Embedding(input_dim=self.max_features, output_dim=hp.Int('embedding_dim', min_value=128, max_value=128, step=32)))
        model.add(GRU(units=hp.Int('units', min_value=64, max_value=256, step=32),   #https://keras.io/api/layers/recurrent_layers/gru/
                      dropout=hp.Float('dropout', min_value=0.1, max_value=0.4, step=0.1),
                      recurrent_dropout=hp.Float('dropout', min_value=0.1, max_value=0.4, step=0.1),
                      input_shape=(self.max_sequence_length, 1)))
        model.add(Dense(1, activation="sigmoid"))

        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])),
                      loss=self.loss_function,
                      metrics=["accuracy"])
        return model

    def build_model(self,hp):
        model_name = hp.Choice('model_name', values=['GRU', 'BidirectionalLSTM', 'LSTM'])
        if model_name == 'LSTM':
            return self.build_lstm_model(hp)
        elif model_name == 'BidirectionalLSTM':
            return self.build_bidirectional_lstm_model(hp)
        elif model_name == 'GRU':
            return self.build_gru_model(hp)

    def fit(self, hp, model, *args, **kwargs):
        weight_for_0 = 1. / counts[0]
        weight_for_1 = 1. / counts[1]
        class_weight = {0: weight_for_0, 1: weight_for_1}
        return model.fit(X_train, y_train, validation_data=(X_val, y_val),class_weight=class_weight)

    def summary(self):
        return self.model.summary()

    def save(self, path):
        return self.model.save(path)

    def predict(self, x):
        return self.model.predict(x)

In [40]:

loss = tf.keras.losses.BinaryFocalCrossentropy(
    apply_class_balancing=False,
    alpha=0.25,
    gamma=2.0,
    from_logits=False,
    label_smoothing=0.0,
    axis=-1,
    name='binary_focal_crossentropy'
)
# Define hyperparameters for the tuner
hypermodel_builder = HyperModelBuilderBinary(max_sequence_length=max_sequence_length,
                                       loss_function=loss, max_features=10000)

# Initialize tuner
tuner = kt.BayesianOptimization(hypermodel_builder.build_model,
                        objective='val_loss',
                        max_trials=10,
                        executions_per_trial=1, # The documentation says that makes things faster
                        overwrite=True,
                        max_consecutive_failed_trials=5,
                        directory='tuner_results',
                        project_name='regression_tuning')

# Summarize the search space
tuner.search_space_summary()

Model: "sequential_20"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_25 (Embedding)             │ (None, 1600, 128)           │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_20 (GRU)                         │ (None, 64)                  │          37,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,317,313 (5.03 MB)

 Trainable params: 1,317,313 (5.03 MB)

 Non-trainable params: 0 (0.00 B)

Search space summary
Default search space size: 5
model_name (Choice)
{'default': 'GRU', 'conditions': [], 'values': ['GRU', 'BidirectionalLSTM', 'LSTM'], 'ordered': False}
embedding_dim (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
units (Int)
{'default': None, 'conditions': [], 'min_value': 64, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
dropout (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.4, 'step': 0.1, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.001, 0.0001, 1e-05], 'ordered': True}


##Training & Evaluating

In [42]:
def compute_class_accuracies(X_data, y_data, model):
  # The predicted probabilities for each class
  y_pred_probs = model.predict(X_data)

  # The corresponding predicted labels
  y_pred_labels = np.argmax(y_pred_probs, axis=0)

  # The corresponding true labels
  y_true_labels = np.argmax(y_data, axis=0)

  # Compute confusion matrix
  cm = confusion_matrix(y_true_labels, y_pred_labels)

  # Compute accuracies and weights for each class
  class_accuracies = []
  for class_label in range(len(cm)):
      class_accuracy = cm[class_label, class_label] / np.sum(cm[class_label, :])
      class_accuracies.append(class_accuracy)

  # Print the classification report
  print(classification_report(y_true_labels, y_pred_labels))

  return class_accuracies

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=SEED)

In [44]:
del y
del X
gc.collect()

6921

In [ ]:
if TRAIN_BASELINE:
  baseline_model.fit(X_train, y_train)
  joblib.dump(baseline_model, '/content/DL_Ass3/baseline_model_cla.pkl')
else:
  baseline_model = joblib.load('/content/DL_Ass3/baseline_model_cla.pkl')

In [ ]:
y_pred = baseline_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
  # Print the classification report
print(classification_report(y_test, y_pred))

We can see that the model is only predicting 0

Train or model

In [45]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=True,  random_state=SEED)

In [46]:
_, X_search, _, y_search = train_test_split(X_train, y_train, test_size=0.1, shuffle=True,  random_state=SEED)
X_search, X_searchVal, y_search, y_searchVal = train_test_split(X_search, y_search, test_size=0.2, shuffle=True,  random_state=SEED)

In [47]:
if TRAIN_MODEL:
  # Early stop to prevent overfitting
  stop_early = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=4,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
    start_from_epoch=0,
  )

  # Perform hyperparameter search
  tuner.search(X_search, y_search, validation_data=(X_searchVal, y_searchVal), epochs=1, batch_size=256, callbacks=[stop_early])
  # Get the best hyperparameters
  best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
  # Build the model
  best_model = tuner.hypermodel.build(best_hps)
  # Save the model
  best_model.save('content/DL_Ass3/best_model_cla.keras')
  # Fit the model
  history = best_model.fit(X_train, y_train, epochs=50, batch_size=512, validation_data=(X_val, y_val), callbacks=[stop_early])
  # Save the model
  best_model.save('content/DL_Ass3/best_model_cla.keras')
  plot_history(history)
else:
  best_model = keras.models.load_model('content/DL_Ass3/best_model_cla.keras')


Trial 10 Complete [00h 03m 27s]
val_loss: nan

Best val_loss So Far: 0.042394958436489105
Total elapsed time: 00h 29m 43s


FileNotFoundError: [Errno 2] No such file or directory: 'content/DL_Ass3/best_model_cla.keras'

In [33]:
model = Sequential([Input(shape=(1600,)),(Embedding(input_dim=(10000), output_dim=128)),(GRU(units=256,   #https://keras.io/api/layers/recurrent_layers/gru/
                      dropout=0.1,
                      recurrent_dropout=0.1,
                      input_shape=(1600, 1))), (Dense(1, activation="sigmoid"))])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
                      metrics=["accuracy"])
model.summary()



input_layer = Input(shape=(1600,))

# Add embedding layer
emb = Embedding(input_dim=10000, output_dim=128)(input_layer)

# Add GRU layer
x = GRU(units=256,
        dropout=0.1,
        recurrent_dropout=0.1)(emb)

# Add output layer
head = Dense(1, activation="sigmoid")(x)

# Define model
model = Model(inputs=input_layer, outputs=head)
model.summary()

Model: "sequential_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_21 (Embedding)             │ (None, 1600, 128)           │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_16 (GRU)                         │ (None, 256)                 │         296,448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,576,705 (6.01 MB)

 Trainable params: 1,576,705 (6.01 MB)

 Non-trainable params: 0 (0.00 B)

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)           │ (None, 1600)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_22 (Embedding)             │ (None, 1600, 128)           │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_17 (GRU)                         │ (None, 256)                 │         296,448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,576,705 (6.01 MB)

 Trainable params: 1,576,705 (6.01 MB)

 Non-trainable params: 0 (0.00 B)